# Feature Scaling Validation

This notebook validates the feature scaling by:
1. Loading the saved scaler (created by `scripts/apply_scaling.py`)
2. Loading the scaled dataset (or applying scaling)
3. Verifying scaling properties for each feature
4. Checking for data leakage using the `split_group` column


In [43]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent/"src"))

from features.scaling import FeatureScaler, NUMERICAL_FEATURES
from sklearn.preprocessing import StandardScaler, MinMaxScaler


## 1. Load Scaler and Scaled Dataset


In [44]:
# Load scaler
models_dir = Path('../models')
scaler_path = models_dir / 'scaler.joblib'

if not scaler_path.exists():
    raise FileNotFoundError("No scaler found. Run 'scripts/apply_scaling.py' first")

scaler = FeatureScaler.load(str(scaler_path))
print(f"Scaler loaded: {scaler_path.name}")
print(f"    Method: {scaler.method.upper()}")
print(f"    Skew threshold: {scaler.skew_threshold}")
print(f"    Features: {len(NUMERICAL_FEATURES)}")
print("="*70)

# Determine scaler type for each feature (for 'auto' method)
scaler_types = {}
if scaler.method == 'auto':
    for feature in NUMERICAL_FEATURES:
        print(f"Feature: {feature} - Scaler: {scaler.scalers[feature]}")
else:
# Print the method field otherwise
    print(scaler.method)

# Load scaled dataset if it exists, otherwise load original and apply scaling
scaled_file = Path('../data/processed/openfoodfacts_scaled.csv')
if scaled_file.exists():
    df_scaled = pd.read_csv(scaled_file)
    print(f"Loaded Scaled Dataset with shape: {df_scaled.shape}")
    
    # Check if split_group column exists
    if 'split_group' in df_scaled.columns:
        print("="*70)
        print(f"Found 'split_group' column for train/test tracking")
    else:
        print("="*70)
        print(f"Warning: 'split_group' column not found")
else:
    print(f"\n No Scaled Dataset found. Run 'scripts/apply_scaling.py' first")


Scaler loaded: scaler.joblib
    Method: AUTO
    Skew threshold: 1.0
    Features: 9
Feature: energy-kcal_100g - Scaler: StandardScaler()
Feature: fat_100g - Scaler: MinMaxScaler()
Feature: saturated-fat_100g - Scaler: MinMaxScaler()
Feature: carbohydrates_100g - Scaler: StandardScaler()
Feature: sugars_100g - Scaler: MinMaxScaler()
Feature: fiber_100g - Scaler: MinMaxScaler()
Feature: proteins_100g - Scaler: MinMaxScaler()
Feature: salt_100g - Scaler: MinMaxScaler()
Feature: additives_n - Scaler: MinMaxScaler()
Loaded Scaled Dataset with shape: (96132, 21)
Found 'split_group' column for train/test tracking


## 2. Split Data Using split_group Column


In [45]:
# Split using split_group column if available
if 'split_group' in df_scaled.columns:
    train_mask = df_scaled['split_group'] == 'train'
    X_train_scaled = df_scaled[train_mask].drop(columns=['split_group'], errors='ignore')
    X_test_scaled = df_scaled[~train_mask].drop(columns=['split_group'], errors='ignore')
    print(f"✓ Using 'split_group' column for splitting")
    print(f"  Train: {len(X_train_scaled):,} | Test: {len(X_test_scaled):,}")
else:
    # Fallback: recreate split (not ideal, but works)
    print("⚠ 'split_group' not found. Recreating split...")
    from sklearn.model_selection import train_test_split
    df_original = pd.read_csv('../data/processed/openfoodfacts_cleaned.csv')
    X = df_scaled.drop(columns=['nutriscore_grade', 'code'])
    y = df_original['nutriscore_grade']
    X_train_scaled, X_test_scaled, _, _ = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"  Train: {len(X_train_scaled):,} | Test: {len(X_test_scaled):,}")


✓ Using 'split_group' column for splitting
  Train: 76,905 | Test: 19,227


## 3. Verify Scaling Properties


In [46]:
summary_data = []

for col in NUMERICAL_FEATURES:
    series = X_train_scaled[col]
    scaler_type = scaler.scalers[col].__class__.__name__
    
    if scaler_type == 'MinMaxScaler':
        # In MinMax we just check the bounds
        check = "Min: " + str(round(series.min(), 2)) + " | Max: " + str(round(series.max(), 2))  
    elif scaler_type == 'StandardScaler':
        # In Standard we check the mean and std
        check = "Mean: " + str(round(series.mean(), 2)) + " | Std: " + str(round(series.std(), 2))
    summary_data.append({
        'Feature': col,
        'Scaler Used': scaler_type,
        'Sanity Check': check
    })

check_df = pd.DataFrame(summary_data).set_index('Feature')
print(f"Global Strategy: {scaler.method.upper()}\n")
display(check_df) 

Global Strategy: AUTO



,Scaler Used,Sanity Check
Feature,,
energy-kcal_100g,StandardScaler,Mean: 0.0 | Std: 1.0
fat_100g,MinMaxScaler,Min: 0.0 | Max: 1.0
saturated-fat_100g,MinMaxScaler,Min: 0.0 | Max: 1.0
carbohydrates_100g,StandardScaler,Mean: 0.0 | Std: 1.0
sugars_100g,MinMaxScaler,Min: 0.0 | Max: 1.0
fiber_100g,MinMaxScaler,Min: 0.0 | Max: 1.0
proteins_100g,MinMaxScaler,Min: 0.0 | Max: 1.0
salt_100g,MinMaxScaler,Min: 0.0 | Max: 1.0
additives_n,MinMaxScaler,Min: 0.0 | Max: 1.0


## 4. Check Data Leakage Prevention


In [47]:
data = []

for col in NUMERICAL_FEATURES:
    s = scaler.scalers[col].__class__.__name__
    d = X_test_scaled[col]
    
    # Check if Test data looks "Suspiciously Perfect" (Evidence of Leakage)
    if s == 'StandardScaler':
        # Suspicious if Mean almost 0 AND Std almost 1
        is_leaked = np.isclose(d.mean(), 0, atol=0.05) and np.isclose(d.std(), 1, atol=0.02)
        stats = "Mean: " + str(round(d.mean(), 2)) + " | Std: " + str(round(d.std(), 2))
        
    elif s == 'MinMaxScaler':
        # Suspicious if Min almost 0 AND Max almost 1
        is_leaked = np.isclose(d.min(), 0, atol=0.01) and np.isclose(d.max(), 1, atol=0.01)
        stats = "Min: " + str(round(d.min(), 2)) + " | Max: " + str(round(d.max(), 2))
    data.append({
        "Feature": col,
        "Scaler": type(s).__name__,
        "Test Set Stats": stats,
        "Leakage Detected": "⚠️ YES" if is_leaked else "✅ NO"
    })

print("DATA LEAKAGE CHECK (Target: NO for all rows)")
display(pd.DataFrame(data))

DATA LEAKAGE CHECK (Target: NO for all rows)


,Feature,Scaler,Test Set Stats,Leakage Detected
0,energy-kcal_100g,str,Mean: -0.0 | Std: 1.07,✅ NO
1,fat_100g,str,Min: 0.0 | Max: 1.0,⚠️ YES
2,saturated-fat_100g,str,Min: 0.0 | Max: 0.7,✅ NO
3,carbohydrates_100g,str,Mean: -0.01 | Std: 1.0,⚠️ YES
4,sugars_100g,str,Min: 0.0 | Max: 1.0,⚠️ YES
5,fiber_100g,str,Min: 0.0 | Max: 0.92,✅ NO
6,proteins_100g,str,Min: 0.0 | Max: 0.9,✅ NO
7,salt_100g,str,Min: 0.0 | Max: 0.99,✅ NO
8,additives_n,str,Min: 0.0 | Max: 0.74,✅ NO


## Observation on Data Leakage Check: False Positives

The **"Leakage Detected"** warnings reported for specific features (e.g. `sugars_100g`, `fat_100g`, `carbohydrates_100g`) are **false positives** and can be safely ignored.

### Reasoning

This behavior stems from the **physical nature of nutritional data**, where features are constrained by **hard, known boundaries** (e.g. values between 0 g and 100 g).

Because the dataset is sufficiently large, **both the Training Set and the Test Set fully cover the physical range** of these variables, including values at or very close to the minimum and maximum.

As a result:

- `MinMaxScaler` maps the Test Set bounds to **0.0** and **1.0**
- This behavior **mirrors the Training Set scaling**
- **No information from the Test Set is leaked into the Training Set**

### Conclusion

The apparent leakage is an **artifact of bounded features**, not a modeling flaw.  
The **train/test separation integrity is fully preserved**, and the warnings do **not** indicate real data leakage.

This can be seen executing the cell below:

In [51]:
bounds_train = X_train_scaled[NUMERICAL_FEATURES].agg(['min', 'max'])
bounds_test = X_test_scaled[NUMERICAL_FEATURES].agg(['min', 'max'])

is_saturated = (bounds_train == bounds_test)

print("Features with same bounds (Causa of false 'Leakage Detected'):")
display(is_saturated.loc[:, is_saturated.all()])

Features with same bounds (Causa of false 'Leakage Detected'):


,fat_100g,carbohydrates_100g,sugars_100g
min,True,True,True
max,True,True,True


## 5. Summary


In [54]:
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"\nScaler loaded: {scaler_path.name}")
print(f"Method: {scaler.method.upper()}")
if scaler.method == 'auto':
    print(f"Skew threshold: {scaler.skew_threshold}")
    print(f"\nScaler distribution:")
    from collections import Counter
    type_counts = Counter(scaler_types.values())
    for stype, count in type_counts.items():
        print(f"  {stype}: {count} features")
print(f"Features scaled: {len(NUMERICAL_FEATURES)}")
print(f"\nDataset:")
print(f"  Train: {len(X_train_scaled):,} samples")
print(f"  Test:  {len(X_test_scaled):,} samples")
print(f"\n Scaling validation completed")
print(f" Ready for model training")
print(f" No leakage detected")



VALIDATION SUMMARY

Scaler loaded: scaler.joblib
Method: AUTO
Skew threshold: 1.0

Scaler distribution:
Features scaled: 9

Dataset:
  Train: 76,905 samples
  Test:  19,227 samples

 Scaling validation completed
 Ready for model training
 No leakage detected
